In [1]:
import sys
from lib.spark_session import get_spark_session
from lib.utils import  get_string_cols,get_rundate
from lib.aws_s3 import  archive_landing_object
from lib.job_control import insert_log, get_max_timestamp
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import current_timestamp, lit
from datetime import datetime
from delta import DeltaTable

In [2]:
# JOB Parameters
rundate = get_rundate()
schema_name = "edw_ld"
table_name = "dim_product_ld"
table_full_name = f"{schema_name}.{table_name}" 
landing_file_name = f"product_{rundate}.csv"
landing_file_path = f"s3a://deltalake12/dw-with-pyspark/landing/product/"
print("SPARK_APP: JOB triggered for rundate - " + rundate)

SPARK_APP: JOB triggered for rundate - 20220101


In [3]:
spark: SparkSession = get_spark_session(f"Landing load - {table_full_name}")
print("SPARK_APP: Spark UI - " + spark.sparkContext.uiWebUrl)

SPARK_APP: Spark UI - http://03205cdd01e3:4040


In [4]:
# Spark Configs
spark.conf.set("spark.sql.shuffle.partitions", 8)
spark.conf.set("spark.sql.parquet.mergeSchema", True)

In [5]:
# Create Raw dataframe and get col info
df_raw = spark.read.format("csv").option("header",True).load(f"{landing_file_path}/{landing_file_name}")
print("SPARK_APP: Printing Raw Schema --")
df_raw.printSchema()

# Get Landing count
print("SPARK_APP: Landing data count - " + str(df_raw.count()))

SPARK_APP: Printing Raw Schema --
root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- type: string (nullable = true)
 |-- flavor: string (nullable = true)
 |-- size: string (nullable = true)
 |-- price: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- expiration_date: string (nullable = true)
 |-- image_url: string (nullable = true)

SPARK_APP: Landing data count - 14


In [6]:
# Cast all columns to String
df_casted = df_raw.selectExpr(get_string_cols(spark, df_raw))
print("SPARK_APP: Casted all columns to String")

# Add audit columns
df_ld = df_casted.withColumn("insert_dt", current_timestamp()) \
    .withColumn("rundate", lit(rundate))
print("SPARK_APP: Added AUDIT column")

# Get Final Layer count
print("SPARK_APP: Final layer data count - " + str(df_ld.count()))
print("SPARK_APP: Printing Landing Schema --")
df_ld.printSchema()

SPARK_APP: Casted all columns to String
SPARK_APP: Added AUDIT column
SPARK_APP: Final layer data count - 14
SPARK_APP: Printing Landing Schema --
root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- type: string (nullable = true)
 |-- flavor: string (nullable = true)
 |-- size: string (nullable = true)
 |-- price: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- expiration_date: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- insert_dt: timestamp (nullable = false)
 |-- rundate: string (nullable = false)



In [7]:
# Write the data to landing layer checking if table exists
if get_max_timestamp(spark, schema_name, table_name) != "1900-01-01 00:00:00.000000":
    df_ld.write \
        .format("delta") \
        .mode("append") \
        .saveAsTable(table_full_name)
else:
    df_ld.write \
        .format("delta") \
        .mode("overwrite") \
        .saveAsTable(table_full_name)

print("SPARK_APP: Data written to landing layer")

SPARK_APP: Data written to landing layer


In [8]:
# Archive the landing file
if archive_landing_object(landing_file_name, "store"):
    print(f"SPARK_APP: Landing file {landing_file_name} archived")
else:
    print(f"SPARK_APP: ERROR - Landing file {landing_file_name} not archived. Please Archive the file manually")

Unable to locate credentials
SPARK_APP: ERROR - Landing file product_20220101.csv not archived. Please Archive the file manually


In [9]:
insert_log(spark, schema_name, table_name, datetime.now(), rundate)
print("SPARK_APP: Update JOB Control Log")

SPARK_APP: Update JOB Control Log


In [10]:
spark.sql(f"select * from edw.job_control where table_name = '{table_name}' order by insert_dt desc").show(truncate=False)

+-----------+--------------+--------------------------+--------+--------------------------+
|schema_name|table_name    |max_timestamp             |rundate |insert_dt                 |
+-----------+--------------+--------------------------+--------+--------------------------+
|edw_ld     |dim_product_ld|2024-06-01 07:48:19.310747|20220101|2024-06-01 07:48:20.243416|
+-----------+--------------+--------------------------+--------+--------------------------+



In [11]:
# Get the logs from delta table version
dt = DeltaTable.forName(spark, table_full_name)
dt.history().limit(1).select("version","operationMetrics.executionTimeMs", 
                                 "operationMetrics.numTargetRowsInserted",
                                "operationMetrics.numTargetRowsUpdated",
                                "operationMetrics.numOutputRows").show(1, False)

+-------+---------------+---------------------+--------------------+-------------+
|version|executionTimeMs|numTargetRowsInserted|numTargetRowsUpdated|numOutputRows|
+-------+---------------+---------------------+--------------------+-------------+
|0      |null           |null                 |null                |14           |
+-------+---------------+---------------------+--------------------+-------------+



In [12]:
dt.generate("symlink_format_manifest")
print("SPARK_APP: Symlink Manifest file generated")

SPARK_APP: Symlink Manifest file generated


In [14]:
spark.sql("select * from edw_ld.dim_product_ld").show()

+----------+--------------------+-----------------+----+-------+-------+-----+--------+---------------+--------------------+--------------------+--------+
|product_id|        product_name|            brand|type| flavor|   size|price|quantity|expiration_date|           image_url|           insert_dt| rundate|
+----------+--------------------+-----------------+----+-------+-------+-----+--------+---------------+--------------------+--------------------+--------+
|      P001|     Purina Pro Plan|           Purina| Dry|Chicken|  5 kgs|Rs 20|      50|     31-12-2024|https://www.examp...|2024-06-01 07:47:...|20220101|
|      P002| Hill's Science Diet|           Hill's| Dry|   Beef| 15 kgs|Rs 50|      25|     31-12-2024|https://www.examp...|2024-06-01 07:47:...|20220101|
|      P003|Iams ProActive He...|             Iams| Dry|   Lamb| 10 kgs|Rs 40|      35|     31-12-2024|https://www.examp...|2024-06-01 07:47:...|20220101|
|      P004|Blue Buffalo Wild...|     Blue Buffalo| Wet| Salmon|12 can

In [15]:
spark.stop()